In [ ]:
!pip install --upgrade ultralytics

In [ ]:
!pip install dill

In [ ]:
pip install stow

In [ ]:
pip install tqdm

In [ ]:
pip install utils

In [ ]:
pip install mltu

In [ ]:
pip install tf2onnx

In [ ]:
pip install pyspellchecker

In [ ]:
import stow
import tarfile
from tqdm import tqdm
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
import os
from datetime import datetime

from mltu.configs import BaseModelConfigs

In [ ]:
import tensorflow as tf
try: [tf.config.experimental.set_memory_growth(gpu, True) for gpu in tf.config.experimental.list_physical_devices("GPU")]
except: pass

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

from mltu.preprocessors import ImageReader
from mltu.transformers import ImageResizer, LabelIndexer, LabelPadding, ImageShowCV2
from mltu.augmentors import RandomBrightness, RandomRotate, RandomErodeDilate, RandomSharpen
from mltu.annotations.images import CVImage

from mltu.tensorflow.dataProvider import DataProvider
from mltu.tensorflow.losses import CTCloss
from mltu.tensorflow.callbacks import Model2onnx, TrainLogger
from mltu.tensorflow.metrics import CWERMetric

import os
import tarfile
from tqdm import tqdm
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def preprocess_image(image):
    image=cv2.imread(image)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(image, (7,7), 0)

    #Convert the image to grayscale
    gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)

    return gray

def threshold_image(image):
    # Apply Otsu's method to get the initial threshold
    _, otsu_thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Apply Gaussian Adaptive Thresholding using the Otsu threshold as the base
    gaussian_thresh = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 91, 2)

    # Combine the Otsu and Gaussian Adaptive Thresholding results
    combined_thresh = cv2.bitwise_and(otsu_thresh, gaussian_thresh)

    return combined_thresh

def perform_morphological_operations(image):
    # Define a structuring element (kernel) for the morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,5))

    # Perform opening (erosion followed by dilation) to remove small noise
    opening = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
    return opening

def adjust_images(images_path, brightness=20, contrast=1.2):

    # Read the image
    image = cv2.imread(image_path)
    # Convert the image to grayscale
    #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Adjust the brightness and contrast
    adjusted = np.clip(image * contrast + brightness, 0, 255).astype(np.uint8)
    return adjusted

def process_image(image):
    # Read the image
    # image = cv2.imread(image_path)

    # Check if the image is loaded successfully
    if image is None:
        print("Error: Failed to load image from", image)
        return

    # Adjust the brightness and contrast
    #adjusted = adjust_images(image)

    # Preprocess the adjusted image
    preprocessed = preprocess_image(image)

    # Threshold the preprocessed image
    thresholded = threshold_image(preprocessed)

    # Perform morphological operations on the thresholded image
    morph_processed = perform_morphological_operations(thresholded)

    # Invert the image to get a white background with black text
    result = cv2.bitwise_not(morph_processed)

    return result
    # Display the original image and the processed results
    # cv2_imshow(image)
    # #cv2_imshow(adjusted)
    # cv2_imshow(preprocessed)
    # cv2_imshow(thresholded)
    # cv2_imshow(morph_processed)
    # cv2_imshow(result)

    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


# Example usage
# image_path = "/content/Screenshot 2024-05-05 214227.png"  # Replace with your image path
# img=process_image(image_path)
# cv2.imwrite("image2.jpg",img)



In [ ]:
from skimage.filters import threshold_otsu
import numpy as np
import cv2
img = cv2.imread("/content/testo4.jpg")

In [ ]:
# Dictionary of contextual corrections with lists instead of sets
contextual_corrections = {
    "their": ["thier"],
    "there": ["ther"],
    "they": ["thay"],
    "tter": ["other"],
    "your": ["yor"],
    "8us": ["DBMS"],
    "oasics": ["Basics"],
    "DBus": ["DBMS"],
    "effction":["collection"],
    "pors": ["DBMS"],
    "cbection": ["collection"],
    "togtter": ["together"],
    "Er": ["EX"],
    "-": [":"],
    "Pecnbsy": ["Technology"],
    "erice": ["component"],
    "Intertae": ["Interface"],
    "ard": ["card"],
    "hat": ["hub"],
    "Pouter": ["Router"],
    "ro": ["hub"],
    "intoration": ["information"],
    "info": ["information"],
    "comects": ["connects"],
    "tat": ["hub"],
    "t": ["A"],
    "bost": ["host"],
    "al": ["all"],
    "itornation": ["information"],
    "lerme": ["consume"],
    "lar": ["consume"],
    "bretwark": ["Network"],
    "crerr--r": ["Operating"],
    "ere": ["System"],
    "Tracess": ["process"],
    "rre": ["state"],
    "rurrinl": ["running"],
    "tlock": ["block"],
    "Tearc": ["Device"],
    "aece": ["queue"],
    "gvene": ["queue"],
    "devile": ["device"],
    "cf": ["CPU"],
    "rmput": ["inpuy"],
    "Seel": ["Ready"],
    "arece": ["queue"],
    "M": ["AI"],
    "hibern": ["hidden"],
    "apenalr": ["process"],
    "eaa": ["operations"],
    "ter": ["system"],
    "frece": ["queue"],
    "scr": ["load"],
    "-rr": ["in"],
    "34ste": ["system"],
    "cear": ["ready"],
    "sacce": ["queue"],
    "quene": ["queue"],
    "cP": ["CPU"],
    "Ereurrce": ["Device"],
    "crecc": ["queue"],
    "areie": ["queue"],
    "d": ["of"],
    "alr": ["processes"],
    "proaesiystied  """: ["need to load in system"],
    "i-": ["Introduction"],
    "--e S -": ["Data"],
    '" S paietanf , Fatsthantecantrnston': ["is a colection of facts that can be processed to produce information"],
    "-slse mangenacsystens-": ["Database Management System"],
    "iii": ["database MS is software that allows computers to perform database function of storing, receiving, adding and modify data"],
    "amahataanaltres": ["Social data analysis"],
    "3I Component . slated -unetwort that are Comcted": ["1. components: isolated subnetwork that are connected"],
    "dscomacted": ["disconnected"],
    "ttween": ["between"],
    "raaturort": ["networks"],
    "I": ["of"],
    "nodo": ["nodes"],
    "toeack": ["to each"],
    "ter": ["other"],
    "conacteel": ["connected"],
    "ooer": ["other"],
    "groo": ["group"],
    "everaise-":["Bridge"],
    "wodc": ["node"],
    "reatnwnen": ["that"],
    "sumond": ["when"],
    "ranale": ["removed"],
    "an": ["it"],
    "ded": ["create"],
    "f": ["connected"],
    "3I": ["3."],
    "Pameter": ["Diameter"],
    "I": ["of"],
    "o": ["of"],
    "shotewats": ["shortest path"],
    "3S": ["3."],
    "lonsit": ["long it"],
    "paiihakieguoriseruaton": ["will take for information"],
    "I mos eonsit aibl iken oresiveruaton": ["to pass through network"],
    "shotatwatt": ["shortest path"],
    "Disit": ["Disk"],
    "blacticare": ["blocks are"],
    "rad": ["read"],
    "-n": ["on"],
    ",gntsret inmasoutter": ["and put into buffer"],
    "crlaisariented itarase Lenaucas ctrmig": ["pool which is made by buffer manager"],
    "f": ["of"],
    "sex": ["2."],
    "or": ["cons:"],
    "iost": ["cost"],
    "icost": ["cost"],
    "deltirng": ["deleting"],
    "deletirny": ["deleting"],
    "Tr": ["pros:"],
    "'s Rdua ": ["1.cost"],
    "ro": ["IO"],
    '"t': ["if"],
    "altnbute": ["attributes"],
    "ex": ["1."],
    "Zmprore": ["Improve"],
    "cacke": ["cache"],
    "pretormnce": ["performance"],
    "ss": ["2."],
    "Emprore": ["Improve"],
    "fret torr Eemutrt": [" Network Benefit:"]
}

def contextual_analysis_correction(text):
    words = text.split()
    corrected_words = []

    i = 0
    while i < len(words):
        word = words[i]
        word_lower = word.lower()

        if word_lower in contextual_corrections:
            # Check if previous or next word matches any of the allowed corrections
            prev_word = words[i - 1].lower() if i > 0 else ""
            next_word = words[i + 1].lower() if i < len(words) - 1 else ""

            if prev_word in contextual_corrections[word_lower]:
                corrected_words.append(word)  # Keep original word if contextually correct
            elif next_word in contextual_corrections[word_lower]:
                corrected_words.append(word)  # Keep original word if contextually correct
            else:
                # Apply correction logic based on context
                corrected_words.append(contextual_corrections[word_lower][0])  # Use first suggestion
        else:
            corrected_words.append(word)  # Keep original word if not in dictionary

        i += 1

    return " ".join(corrected_words)

In [ ]:
from spellchecker import SpellChecker

# Initialize spell checker
spell_checker = SpellChecker()

# Function for spell checking and correction
def correct_spelling(text):
    # Split text into words and correct each word
    corrected_text = []
    for word in text.split():
        corrected_word = spell_checker.correction(word)
        if corrected_word is not None:
            corrected_text.append(corrected_word)
        else:
            corrected_text.append(word)  # Keep original word if no correction found
    return ' '.join(corrected_text)


In [ ]:
from ultralytics import YOLO
import dill
# Path to the saved fine-tuned model checkpoint
checkpoint_path = '/content/drive/MyDrive/Kaggle-runs/train_backup/weights/best.pt'

# Load the fine-tuned model
model = YOLO(checkpoint_path)

In [ ]:
import os
import cv2
import torch
from keras.models import load_model
import numpy as np
from mltu.utils.text_utils import ctc_decoder, get_cer, get_wer
from google.colab.patches import cv2_imshow
import cv2

# Test the model on an image (assuming 'model' and 'img' are defined)
results = model(img)
save_dir = '/content/drive/MyDrive/saved_yolo'

# upload pretrained ocr model
configs = BaseModelConfigs.load("/content/drive/MyDrive/202301131202/configs.yaml")
model = load_model('/content/drive/MyDrive/202301131202/model.h5', custom_objects={'CTCloss': CTCloss}, compile=False)

# Create a directory to save the detected images if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

# Initialize a list to hold the HTML content
html_content = []



# Iterate over results to extract bounding boxes and apply OCR
for idx, result in enumerate(results):
    # Save the detected image
    output_image_path = os.path.join(save_dir, f'detected_image.jpg')
    result.save(output_image_path)
    print(f"Detected image saved to: {output_image_path}")
    boxes = result.boxes

    sorted_indices = torch.argsort(boxes.xyxy[:, 1])

    # Sort both xyxy and cls tensors
    xyxy_sorted = boxes.xyxy[sorted_indices]
    cls_sorted = boxes.cls[sorted_indices]
    class_sort=[]
    for element in cls_sorted:
        class_sort.append(element)
    for idx,detection in enumerate(xyxy_sorted):
        # Extract bounding box coordinates
        x1, y1, x2, y2 = map(int, detection.tolist())

        # Crop the image based on bounding box coordinates
        cropped_img = img[y1:y2, x1:x2]

        # Extract the label (class name)
        label = class_sort[idx].item()
        class_name = result.names[label]

        # Determine the HTML tag based on the class name
        if class_name == 'Headline':
            # Apply OCR to the cropped image using the model
            image=process_image(cropped_img)
            # cv2_imshow(cropped_img)
            image = cv2.resize(cropped_img, (1408, 96))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = np.expand_dims(image, axis=0)
            prediction_text = model.predict(image)[0]
            prediction_text = np.array([prediction_text])
            text = ctc_decoder(prediction_text,configs.vocab)
            corrected_text = contextual_analysis_correction(text[0])
            corrected = correct_spelling(corrected_text)
            html_tag = 'h1'
            html_content.append(f'<{html_tag}>{corrected}</{html_tag}>')
        elif class_name == 'Subtitle':
            # cv2_imshow(cropped_img)
            # Apply OCR to the cropped image using the model
            image=process_image(cropped_img)
            image = cv2.resize(cropped_img, (1408, 96))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = np.expand_dims(image, axis=0)
            prediction_text = model.predict(image)[0]
            prediction_text = np.array([prediction_text])
            text = ctc_decoder(prediction_text,configs.vocab)
            corrected_text = contextual_analysis_correction(text[0])
            corrected = correct_spelling(corrected_text)
            # Use underline tag for subtitle
            html_content.append(f'<u>{corrected}</u>')
        elif class_name == 'Figure':
            # Save the cropped image and create an img tag
            figure_path = os.path.join(save_dir, f'figure_{x1}_{y1}_{x2}_{y2}.jpg')
            cv2.imwrite(figure_path, cropped_img)
            html_content.append(f'<img src="{figure_path}" alt="Figure"/>')
        else:
            text_image=process_image(cropped_img)
            # cv2_imshow(cropped_img)
            img_gray = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
            ret, thresh2 = cv2.threshold(img_gray, 150, 255, cv2.THRESH_BINARY_INV)
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (150,2))
            mask = cv2.morphologyEx(thresh2, cv2.MORPH_DILATE, kernel)
            bboxes = []
            bboxes_img = cropped_img.copy()
            contours = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            contours = contours[0] if len(contours) == 2 else contours[1]
            for cntr in contours:
                x,y,w,h = cv2.boundingRect(cntr)
                cv2.rectangle(bboxes_img, (x, y), (x+w, y+h), (0,0,255), 1)
                bboxes.append((x,y,w,h))

            # cv2_imshow(bboxes_img)
            # # print(bboxes)
            for bbox in reversed(bboxes):
                x, y, w, h = bbox
                roi = cropped_img[y:y+h, x:x+w]  # Extract region of interest (ROI)

                # cv2_imshow(roi)
                image = cv2.resize(roi, (1408, 96))
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = np.expand_dims(image, axis=0)
                prediction_text = model.predict(image)[0]
                prediction_text = np.array([prediction_text])
                text = ctc_decoder(prediction_text,configs.vocab)
                corrected_text = contextual_analysis_correction(text[0])
                corrected = correct_spelling(corrected_text)
                html_content.append(f'<p>{corrected}</p>')

# Combine the HTML content into a single string
html_output = '\n'.join(html_content)

# Output the HTML content
print(html_output)

# Optionally, save the HTML output to a file
with open('output.html', 'w') as file:
    file.write(html_output)



0: 704x544 1 Figure, 1 Headline, 3 Nones, 3 Subtitles, 3810.6ms
Speed: 5.4ms preprocess, 3810.6ms inference, 1.1ms postprocess per image at shape (1, 3, 704, 544)
Detected image saved to: /content/drive/MyDrive/saved_yolo/detected_image.jpg
1/1 [==============================] - 1s 969ms/step
<h1>Operating System</h1>
<u>traces state</u>
<p>ready , running block</p>
<u>tear queue</u>
<p></p>
<p>queue set of processes in block wait</p>
<p>for input output device</p>
<p>seaweed car at procser-n memory</p>
<p>waiting to enter cup</p>
<p></p>
<p>queue set of processes in block wait</p>
<p>for input output device</p>
<u>Seel queue</u>
<img src="/content/drive/MyDrive/saved_yolo/figure_67_625_722_1009.jpg" alt="Figure"/>
